# 量子テレポーテーション


Kifumi Numata (Aug 01, 2022)

In [ ]:
# Qiskitライブラリーを導入
from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_multivector

import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 3量子ビット回路を用意
qc = QuantumCircuit(3,3)    

# Aliceのもつ暗号の量子状態ψを今回はRxで作ります
qc.rx(np.pi/3,0)
qc.barrier()    #回路を見やすくするために入れます

# 回路を描画
qc.draw(output="mpl")

In [ ]:
sim = Aer.get_backend('aer_simulator')
qc_init = qc.copy()    # 量子回路をコピー
qc_init.save_statevector()    # 測定前の状態ベクトルを保存
out_vector = sim.run(qc_init).result().get_statevector()    # 実行して状態ベクトルを得る
plot_bloch_multivector(out_vector)

In [ ]:
# 量子もつれを作ります
qc.h(1)
qc.cx(1, 2)
qc.barrier()    #回路を見やすくするために入れます

# 回路を描画
qc.draw(output="mpl")

In [ ]:
# AliceがCNOTとHで、自分の量子ビット2つをエンタングルさせ測定します。
qc.cx(0, 1)
qc.h(0)
qc.barrier()
qc.measure(0, 0)
qc.measure(1, 1)

# 回路を描画
qc.draw(output="mpl")

In [ ]:
#Aliceが測定結果をBobに送り、Bobが結果に合わせて操作します
qc.z(2).c_if(0, 1)    # 古典レジスター0の値が1だったらZゲートをq2にかける
qc.x(2).c_if(1, 1)    # 古典レジスター1の値が1だったらXゲートをq2にかける

# 回路を描画
qc.draw(output="mpl")

量子テレポーテーション回路が完成しました。qubit2にqubit0の量子状態($R_x(\pi/3)$)が転送されていることを確認しましょう。

In [ ]:
sim = Aer.get_backend('aer_simulator')
qc.save_statevector()
out_vector = sim.run(qc).result().get_statevector()
plot_bloch_multivector(out_vector)

Bobに暗号の量子状態が転送されたことを確認するために、最後にBobの量子ビットに逆向きのX軸回転を適用して、QASMシミュレーターで実験し、測定結果が0になることを確認します。

In [ ]:
# 3量子ビット回路を用意
qc = QuantumCircuit(3,3)  

# Aliceのもつ未知の量子状態ψをRxで作ります。角度はπ/3にしました。
qc.rx(np.pi/3,0)
qc.barrier()    #回路を見やすくするために入れます

# 量子もつれを作ります
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# AliceがCNOTとHで自分の量子ビット2つをエンタングルさせ測定します。
qc.cx(0, 1)
qc.h(0)
qc.barrier()
qc.measure(0, 0)
qc.measure(1, 1)

#Aliceが測定結果をBobに送り、Bobが結果に合わせて操作します
qc.z(2).c_if(0, 1)    # 古典レジスター0の値が1だったらZゲートをq2にかける
qc.x(2).c_if(1, 1)    # 古典レジスター1の値が1だったらXゲートをq2にかける

# 未知の量子状態ψの逆ゲートをかけて０が測定できるか確かめます
qc.rx(-np.pi/3, 2)    
qc.measure(2, 2)

qc.draw(output="mpl")

In [ ]:
# qasm_simulatorで実行して確認します
backend = BasicAer.get_backend('qasm_simulator')
counts = execute(qc, backend).result().get_counts()
print(counts)

from qiskit.visualization import plot_histogram
plot_histogram(counts)

qiskitのビット配列は右端が0なので、Bobのビット(qubit 2)は左端です。左端のビットが全て0になっていることが確認できましたか？

## 演習

Aliceの送る暗号の量子状態をY軸回転ゲート(ryを使います。角度は自由に決めます。)で作ってみましょう。最後に最後にBobの量子ビットに逆向きのY軸回転を適用して、QASMシミュレーターで実験し、測定結果が0になることを確認します。

## 実デバイスでの実行

実デバイスで上記の実験を行ってみましょう。現在のところ、IBMの量子コンピューターは測定後の操作継続ができないので、少しコードを変えます。Aliceが測定結果をBobに送り、Bobが結果に合わせて操作するコードを部分を制御ゲートに変えています。

In [ ]:
# 3量子ビット、1古典ビットの回路を用意
qc = QuantumCircuit(3, 1)    

# Aliceのもつ未知の量子状態ψをRxで作ります。角度はπ/5にしました。
qc.rx(np.pi/5,0)
qc.barrier()    #回路を見やすくするために入れます

# 量子もつれを作ります
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# AliceがCNOTとHで自分の量子ビット2つをエンタングルさせ測定します。
qc.cx(0, 1)
qc.h(0)
qc.barrier()

# Aliceの状態に合わせてBobが操作します（ここを変えています！）
qc.cz(0,2)    # q0が1だったらq2にZゲートをかける
qc.cx(1,2)    # q1が1だったらq2にZゲートをかける

# 未知の量子状態ψの逆ゲートをかけて０が測定できるか確かめます
qc.rx(-np.pi/5, 2)    
qc.measure(2, 0)

qc.draw(output="mpl")

In [ ]:
# 先にシミュレーターでコードが合っているか確認します
t_qc = transpile(qc, sim)
counts = sim.run(t_qc).result().get_counts()
print(counts)
plot_histogram(counts)

実デバイスのプロバイダー情報を入力します。

In [ ]:
# アカウント情報をロードして、使える量子デバイスを確認します
IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q')   # Openプロバイダーの場合
provider = IBMQ.get_provider(hub='ibm-q-education', group='ibm-3', project='kawasaki-camp')
provider.backends()

In [ ]:
#一番空いているバックエンドを自動的に選択します
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
backend = least_busy(provider.backends(filters=lambda b: b.configuration().n_qubits >= 3 and
                                   not b.configuration().simulator and b.status().operational==True))
print("最も空いているバックエンドは: ", backend)

In [ ]:
t_qc = transpile(qc, backend, optimization_level=3)
job = backend.run(t_qc)    # 上記のバックエンドで実行します
job_monitor(job)  # ジョブの実行状態を確認します

In [ ]:
# 結果を確認します
exp_result = job.result()
exp_counts = exp_result.get_counts(qc)
print(exp_counts)
plot_histogram(exp_counts)

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright